In [53]:
import pandas as pd
import numpy as np

# Cargar el dataset
df = pd.read_excel('DATASET VIH.xlsx', sheet_name='Original')

# Limpieza de nombres de columnas
df.columns = df.columns.str.strip()
df = df.rename(columns={
    'mes_caso': 'mes_num',
    'Población': 'Poblacion',
    'Mes': 'mes_nombre'
})

# Eliminar filas completamente vacías
df = df.dropna(how='all')

# Conversión de columnas numéricas
numeric_cols = ['Semana', 'Año', 'mes_num', 'Poblacion']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Limpieza específica para la columna Estrato 
df['Estrato'] = df['Estrato'].replace(['', 'nan', np.nan], 'No especificado')

# Limpieza del resto de columnas categóricas
df['Nombre Nacionalidad'] = df['Nombre Nacionalidad'].fillna('Desconocido').str.strip().str.upper()
df['Sexo'] = df['Sexo'].fillna('Desconocido').str.strip().str.upper().replace({
    'M': 'MASCULINO', 
    'F': 'FEMENINO'
})
df['Grupo Edad'] = df['Grupo Edad'].fillna('Desconocido').str.strip()
df['mes_nombre'] = df['mes_nombre'].fillna('Desconocido').str.strip()

# Creación de fecha
try:
    # Primero intentamos con el nombre del mes
    df['Fecha'] = pd.to_datetime(
        df['Año'].astype(str) + '-' + df['mes_nombre'] + '-01',
        format='%Y-%B-%d',
        errors='coerce'
    )
    
    # Para los que fallaron, intentamos con el número de mes
    mask = df['Fecha'].isna()
    df.loc[mask, 'Fecha'] = pd.to_datetime(
        df.loc[mask, 'Año'].astype(str) + '-' + 
        df.loc[mask, 'mes_num'].astype(str) + '-01',
        format='%Y-%m-%d',
        errors='coerce'
    )
    
    # Si aún hay NA, llenamos con fecha mínima/máxima
    df['Fecha'] = df['Fecha'].fillna(pd.to_datetime('2012-01-01'))
except Exception as e:
    print(f"Error al crear fecha: {e}")
    # Si falla todo, crear una fecha simple basada en año
    df['Fecha'] = pd.to_datetime(df['Año'].astype(str) + '-01-01')

# Ordenar y resetear índice
df = df.sort_values(['Fecha', 'Semana']).reset_index(drop=True)

# Verificación final
print("\nDataFrame")
print(df[['Semana', 'Año', 'Estrato', 'Nombre Nacionalidad', 'Sexo']].head(10))



DataFrame
   Semana   Año          Estrato Nombre Nacionalidad       Sexo
0       1  2012  No especificado            COLOMBIA  MASCULINO
1       1  2012  No especificado            COLOMBIA  MASCULINO
2       1  2012  No especificado            COLOMBIA  MASCULINO
3       1  2012  No especificado            COLOMBIA  MASCULINO
4       1  2012  No especificado            COLOMBIA  MASCULINO
5       1  2012  No especificado            COLOMBIA  MASCULINO
6       1  2012  No especificado            COLOMBIA  MASCULINO
7       1  2012  No especificado            COLOMBIA  MASCULINO
8       1  2012  No especificado            COLOMBIA  MASCULINO
9       1  2012  No especificado            COLOMBIA  MASCULINO


In [55]:
# Porcentaje de valores desconocidos por columna
print("\nPorcentaje de valores desconocidos:")
print((df.isin(['Desconocido', -1]).mean()*100).round(2))



Porcentaje de valores desconocidos:
Semana                 0.0
Año                    0.0
Nombre Nacionalidad    0.0
Sexo                   0.0
Estrato                0.0
Grupo Edad             0.0
mes_num                0.0
mes_nombre             0.0
Poblacion              0.0
Fecha                  0.0
dtype: float64


In [57]:
# Diagnóstico final de limpieza
print("=== Diagnóstico Final ===")

# 1. Verificación de valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# 2. Verificación de Estrato
print("\nDistribución de Estrato:")
print(df['Estrato'].value_counts(dropna=False))

# 3. Muestra aleatoria para verificar consistencia
print("\nMuestra aleatoria de 5 registros:")
print(df.sample(5, random_state=42))

# 4. Verificación de rangos y valores extremos
print("\nResumen estadístico:")
print(df.describe(include='all'))

=== Diagnóstico Final ===

Valores nulos por columna:
Semana                 0
Año                    0
Nombre Nacionalidad    0
Sexo                   0
Estrato                0
Grupo Edad             0
mes_num                0
mes_nombre             0
Poblacion              0
Fecha                  0
dtype: int64

Distribución de Estrato:
Estrato
No especificado    7347
2.0                3055
3.0                2524
1.0                1131
4.0                 320
5.0                  74
6.0                  38
0.0                   2
Name: count, dtype: int64

Muestra aleatoria de 5 registros:
       Semana   Año Nombre Nacionalidad       Sexo          Estrato  \
5936        5  2018         DESCONOCIDO  MASCULINO  No especificado   
12387      43  2021           VENEZUELA  MASCULINO              1.0   
12079      36  2021            COLOMBIA   FEMENINO              3.0   
2227        5  2015         DESCONOCIDO  MASCULINO  No especificado   
169        10  2012         DESCONOCIDO  

In [67]:
import pandas as pd
from datetime import datetime

# 1. Definir el nombre del archivo
fecha_actual = datetime.now().strftime("%Y%m%d_%H%M")
nombre_archivo = f"DATASET_VIH_LIMPIO_{fecha_actual}.xlsx"

# 2. Guardar el DataFrame en Excel 
try:
    with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
        df.to_excel(writer, 
                   index=False, 
                   sheet_name='Datos VIH')
    
    print(f"✅ Dataset guardado exitosamente como: {nombre_archivo}")
    print(f"📄 Total de registros guardados: {len(df):,}")
    print(f"📊 Columnas guardadas: {list(df.columns)}")
    
except Exception as e:
    print(f"❌ Error al guardar el archivo: {e}")
    print("Posibles soluciones:")
    print("1. Asegúrate que tienes instalado openpyxl: pip install openpyxl")
    print("2. Cierra el archivo Excel si está abierto")
    print("3. Verifica los permisos de escritura en la carpeta")

✅ Dataset guardado exitosamente como: DATASET_VIH_LIMPIO_20250508_1728.xlsx
📄 Total de registros guardados: 14,491
📊 Columnas guardadas: ['Semana', 'Año', 'Nombre Nacionalidad', 'Sexo', 'Estrato', 'Grupo Edad', 'mes_num', 'mes_nombre', 'Poblacion', 'Fecha']
